# Sichi Mistral Local Test\n
\n
This notebook only checks that we can talk to a local Mistral model via Ollama.

: 

In [1]:
%pip install requests

  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached charset_normalizer-3.4.4-cp310-cp310-macosx_10_9_universal2.whl.metadata (37 kB)
  Using cached idna-3.11-py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.6.3-py3-none-any.whl.metadata (6.9 kB)
  Using cached certifi-2026.1.4-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.5-py3-none-any.whl (64 kB)
Using cached charset_normalizer-3.4.4-cp310-cp310-macosx_10_9_universal2.whl (209 kB)
Using cached idna-3.11-py3-none-any.whl (71 kB)
Using cached urllib3-2.6.3-py3-none-any.whl (131 kB)
Using cached certifi-2026.1.4-py3-none-any.whl (152 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5/5 [requests]4/5 [requests]

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: /opt/homebrew/opt/python@3.10/bin/python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
OLLAMA_URL = 'http://localhost:11434/api/chat'
MODEL_NAME = 'mistral'
payload = {
    'model': MODEL_NAME,
    'messages': [
        {'role': 'user', 'content': 'Say hello and confirm you are running locally.'}
    ],
    'stream': False,
}
resp = requests.post(OLLAMA_URL, json=payload, timeout=60)
resp.raise_for_status()
data = resp.json()
print('Connected to model:', MODEL_NAME)
print(data['message']['content'])

Connected to model: mistral
 Hello! Yes, I'm running locally as a text-based AI model in this session. How can I assist you today?


In [4]:
import requests

OLLAMA_URL = "http://localhost:11434/api/chat"
MODEL_NAME = "mistral"

def ask_mistral(prompt: str, system: str = "You are Sichi, a banking customer support assistant.") -> str:
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user", "content": prompt},
        ],
        "stream": False,
    }
    response = requests.post(OLLAMA_URL, json=payload, timeout=60)
    response.raise_for_status()
    return response.json()["message"]["content"]

def ask_and_print(prompt: str, system: str = "You are Sichi, a banking customer support assistant.") -> None:
    print(f"User: {prompt}")
    reply = ask_mistral(prompt, system=system)
    print(f"Sichi: {reply}")

  # Example:
ask_and_print("My card was debited but the merchant says payment failed.")


User: My card was debited but the merchant says payment failed.
Sichi:  I'm sorry to hear about your inconvenience. Let me help you resolve this issue. Here are a few steps we can take:

1. Contact the Merchant: Reach out to the merchant and confirm if they have recorded the transaction as unsuccessful. Provide them with the details of the transaction for better assistance.

2. Check your Account Balance: Verify your account balance to ensure that the funds were indeed debited from your account. If the funds are not deducted, there might be an issue with the merchant's system.

3. Transaction Details: Gather all relevant details of the transaction, such as date, time, amount, and the merchant's name or business ID. This information will help in investigating the problem more effectively.

4. Contact Bank Support: If the funds were debited from your account, but the merchant still claims that the payment failed, please contact our bank support team for further assistance. Our team will 

In [6]:
import json

ALLOWED_CATEGORIES = {
    "failed_transfer",
    "card_debit_dispute",
    "bill_payment_issue",
    "reversal_request",
}

ALL_CATEGORIES = [
    "failed_transfer",
    "card_debit_dispute",
    "bill_payment_issue",
    "reversal_request",
    "loan_request",
    "account_opening",
    "other",
]

def classify_intent(user_text: str) -> str:
    classifier_system = (
        "You are a bank support intent classifier.\n"
        "Return ONLY JSON in this exact format: {\"category\":\"...\"}\n"
        f"Choose exactly one from: {ALL_CATEGORIES}"
    )
    raw = ask_mistral(user_text, system=classifier_system)
    try:
        parsed = json.loads(raw)
        category = parsed.get("category", "other")
        return category if category in ALL_CATEGORIES else "other"
    except Exception:
        return "other"

def ask_sichi(user_text: str) -> str:
    category = classify_intent(user_text)

    if category not in ALLOWED_CATEGORIES:
        return (
            "Thanks for sharing this. This issue is outside the categories I can resolve. "
            "I’ve escalated it to a human agent."
        )

    support_system = (
        "You are Sichi, a banking customer support assistant.\n"
        "Handle only banking transaction support.\n"
        "Be concise and practical."
    )
    return ask_mistral(user_text, system=support_system)

def ask_sichi_and_print(user_text: str) -> None:
    category = classify_intent(user_text)
    reply = ask_sichi(user_text)
    print(f"User: {user_text}")
    print(f"Category: {category}")
    print(f"Sichi: {reply}")

# Example
ask_sichi_and_print("I need a personal loan urgently")

User: I need a personal loan urgently
Category: loan_request
Sichi: Thanks for sharing this. This issue is outside the categories I can resolve. I’ve escalated it to a human agent.
